# TASHO - A model predictive control toolchain for constraint-based task specification of robot motions

#### Presented by: Alejandro Astudillo and Ajay Sathya

Tasho stands for “**TA**sk **S**pecification with receding **HO**rizon control”



| <img src="files/img/overview.svg" width="800"> | 
|:--:| 
| Fig. 1. Tasho's overview |

An implementation of a contour-following MPC using **Tasho**

  
First, we need to import the relevant modules from Tasho and other dependencies (casadi, numpy)

In [ ]:
import os
os.chdir(str(os.getcwd())+"/../../")

from tasho import task_prototype_rockit as tp
from tasho import input_resolution, world_simulator
from tasho import robot as rob
from tasho import MPC
from tasho.utils import geometry
import casadi as cs
import numpy as np

## Robot model

Import the robot object from the robot's repository

In [ ]:
robot_choice = "yumi"
# robot_choice = "kinova"
# robot_choice = "iiwa7"
# robot_choice = "atlas"
# robot_choice = "ur10"

robot = rob.Robot(robot_choice, analytical_derivatives=True)

The robot object includes functions for forward dynamics, inverse dynamics, forward kinematics, and their derivatives, beside variables for joint limits and barycentric parameters.

#### Forward dynamics:

$
\ddot{\mathbf{q}} = M^{-1}(\mathbf{q})\left(\tau - C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} - G(\mathbf{q})\right)
$

In [ ]:
FD = robot.fd
print(FD)

In [ ]:
J_FD = robot.J_fd
print(J_FD)

#### Inverse dynamics:
$
\tau = M(\mathbf{q}) + C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} + G(\mathbf{q})
$

In [ ]:
ID = robot.id
print(ID)

In [ ]:
J_ID = robot.J_id
print(J_ID)

#### Forward kinematics:

In [ ]:
FK = robot.fk
print(FK)

#### Upper and lower bound on joint position:

In [ ]:
print("Upper bound on q: ",robot.joint_ub)

In [ ]:
print("Lower bound on q: ",robot.joint_lb)

#### Upper and lower bound on joint velocity:

In [ ]:
print("Upper bound on q_dot: ",robot.joint_vel_ub)

In [ ]:
print("Lower bound on q_dot: ",robot.joint_vel_lb)

#### Upper and lower bound on joint torque:

In [ ]:
print("Upper bound on tau: ",robot.joint_torque_ub)

In [ ]:
print("Lower bound on tau: ",robot.joint_torque_lb)

#### Upper and lower bound on joint acceleration:

In [ ]:
print("Upper bound on q_ddot: ",robot.joint_acc_ub)

In [ ]:
print("Lower bound on q_ddot: ",robot.joint_acc_lb)

#### Update robot's parameters if needed

In [ ]:
ocp_control = "acceleration_resolved"
# ocp_control = "torque_resolved"

if ocp_control == "acceleration_resolved":
    max_joint_acc = 240 * cs.pi / 180
    robot.set_joint_acceleration_limits(lb=[-max_joint_acc]*robot.ndof, ub=[max_joint_acc]*robot.ndof)
    

In [ ]:
print("Upper bound on q_ddot: ",robot.joint_acc_ub)

In [ ]:
print("Lower bound on q_ddot: ",robot.joint_acc_lb)

## Task spacification

Select prediction horizon and sample time for the MPC execution

In [ ]:
horizon_size = 16
t_mpc = 0.01

Initialize the task context object

In [ ]:
tc = tp.task_context(horizon_size * t_mpc, horizon_steps=horizon_size)

Define the input type of the robot (torque or acceleration) and, consequently, the robot dynamics $f_x(x,u)$ to be used within the task context

In [ ]:
if ocp_control == "acceleration_resolved":
    q, q_dot, q_ddot, q0, q_dot0 = input_resolution.acceleration_resolved(tc, robot, {})
elif ocp_control == "torque_resolved":
    q, q_dot, q_ddot, tau, q0, q_dot0 = input_resolution.torque_resolved(tc, robot, 
                                                                         {"forward_dynamics_constraints": False})

Define path dynamics $f_\zeta(\zeta,\nu)$ based on path-progress variable $s$

$
\begin{aligned}
f_\zeta(\zeta,\nu) & = \begin{bmatrix}0 & 1 \\ 0 & 0 \end{bmatrix}\zeta + \begin{bmatrix}0 \\ 1 \end{bmatrix}\nu,\\
\zeta & = \begin{bmatrix}s \\ \dot{s}\end{bmatrix},\ \ \ \ \ \ \ \nu = \ddot{s},
\end{aligned}
$


In [ ]:
s = tc.create_expression("s", "state", (1, 1))
s_dot = tc.create_expression("s_dot", "state", (1, 1))
s_ddot = tc.create_expression("s_ddot", "control", (1, 1))

tc.set_dynamics(s, s_dot)
tc.set_dynamics(s_dot, s_ddot)

Set $s(0)$ and $\dot{s}(0)$ as parameters

In [ ]:
s0 = tc.create_expression("s0", "parameter", (1, 1))
s_dot0 = tc.create_expression("s_dot0", "parameter", (1, 1))

s_init_con = {"expression": s, "reference": s0}
s_dot_init_con = {"expression": s_dot, "reference": s_dot0}
init_constraints = {"initial_constraints": [s_init_con, s_dot_init_con]}
tc.add_task_constraint(init_constraints)

Add constraints for path-progress variable:   
$0 <= s <= 1$  
$\dot{s} >= 0$

In [ ]:
s_con = {
    "hard": True,
    "lub": True,
    "expression": s,
    "lower_limits": 0,
    "upper_limits": 1,
}
s_dotcon = {
    "hard": True,
    "inequality": True,
    "expression": -s_dot,
    "upper_limits": 0,
}
s_path_constraints = {"path_constraints": [s_con, s_dotcon]}
tc.add_task_constraint(s_path_constraints)

#### Define the contour to be followed

In [ ]:
def contour_path(s):
    ee_fk_init = robot.fk(q_init)[7]
    ee_pos_init = ee_fk_init[:3, 3]
    ee_rot_init = ee_fk_init[:3, :3]

    sdotref = 0.1
    sdot_path = sdotref * (
        5.777783e-13 * s ** 5
        - 34.6153846154 * s ** 4
        + 69.2307692308 * s ** 3
        - 46.7307692308 * s ** 2
        + 12.1153846154 * s
        + 0.0515384615
    )

    a_p = 0.15
    z_p = 0.05
    pos_path = ee_pos_init + cs.vertcat(
        a_p * cs.sin(s * (2 * cs.pi)),
        0,
        a_p * cs.sin(s * (2 * cs.pi)) * cs.cos(s * (2 * cs.pi)),
    )
    rot_path = ee_rot_init
    # rot_path = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])

    return pos_path, rot_path, sdot_path

# Define contour/path based on the path-progress variable s
pos_path, rot_path, sdot_path = contour_path(s)

#### Define end-effector position and orientation error  

$
\begin{aligned}
e_{T}(q,s) & = \begin{bmatrix} p_{ee}(q) - p_{ref}(s) \\ \frac{1}{2}(n_{ref}(s)\times n_{ee}(q) + s_{ref}(s)\times s_{ee}(q) + a_{ref}(s)\times a_{ee}(q)) \end{bmatrix},
\end{aligned}
$   
with $R_{ee} = \left[\begin{matrix} n_{ee} & s_{ee} & a_{ee} \end{matrix}\right]$ being the rotation matrix of the end-effector with respect to the robot joint angles $q$, and $R_{ref} = \left[\begin{matrix} n_{ref} & s_{ref} & a_{ref} \end{matrix}\right]$ the reference rotation matrix w.r.t the path-progress variable $s$  

In [ ]:
def pos_err(q, s):
    ee_fk = robot.fk(q)[7]
    return ee_fk[:3, 3] - pos_path


def rot_err(q, s):
    ee_fk = robot.fk(q)[7]
    ee_rot_n = ee_fk[:3, 0]
    ee_rot_s = ee_fk[:3, 1]
    ee_rot_a = ee_fk[:3, 2]

    path_rot_n = rot_path[:, 0]
    path_rot_s = rot_path[:, 1]
    path_rot_a = rot_path[:, 2]

    return 0.5 * (
        geometry.cross_vec2vec(ee_rot_n, path_rot_n)
        + geometry.cross_vec2vec(ee_rot_s, path_rot_s)
        + geometry.cross_vec2vec(ee_rot_a, path_rot_a)
    )


#### Add the tunnel constraint to the task context 
This constraint is managed as a soft constraint (adding a slack variable $l$ to the OCP)  
$\left\Vert\begin{bmatrix} p_{ee}(q) - p_{ref}(s)\\  \frac{1}{2}\begin{bmatrix}n_{ref}(s)\times n_{ee}(q) + s_{ref}(s)\times s_{ee}(q) + a_{ref}(s)\times a_{ee}(q)\end{bmatrix} \end{bmatrix}\right\Vert^{2}  \leq \rho^{2} - l$

In [ ]:
rho = 0.01

tun_tunnel_con = { 
    "hard": False,
    "inequality": True,
    "expression": cs.vertcat(pos_err(q, s), rot_err(q, s)),
    "upper_limits": rho ** 2,
    "gain": 100,
    "norm": "squaredL2",
}
tunnel_constraints = {"path_constraints": [tun_tunnel_con]}
tc.add_task_constraint(tunnel_constraints)

#### Define objective

Add regularization terms to the objective

In [ ]:
tc.add_regularization(expression=(s_dot - sdot_path), weight=20, norm="L2")
tc.add_regularization(expression=pos_err(q, s), weight=1e-1, norm="L2")
tc.add_regularization(expression=rot_err(q, s), weight=1e-1, norm="L2")

if ocp_control == "torque_resolved":
    tc.add_regularization(
        expression=tau, weight=4e-5, norm="L2", variable_type="control", reference=0
    )
if ocp_control == "acceleration_resolved":
    tc.add_regularization(
        expression=q_ddot,
        weight=1e-3,
        norm="L2",
        variable_type="control",
        reference=0,
    )
tc.add_regularization(
    expression=s_ddot, weight=4e-5, norm="L2", variable_type="control", reference=0
)

tc.add_regularization(
    expression=q[0:8], weight=1e-2, norm="L2", variable_type="state", reference=0
)
tc.add_regularization(
    expression=q_dot[0:8], weight=1e-2, norm="L2", variable_type="state", reference=0
)

Add terms at $t_f$ to the objective

In [ ]:
tc.add_objective(
    tc.ocp.at_tf(
        1e-5
        * cs.sumsqr(
            cs.vertcat(
                1e-2 * q[0:8],
                10 * q_dot[0:8],
                1e-2 * (1 - s),
                10 * s_dot,
                10 * pos_err(q, s),
                10 * rot_err(q, s),
            )
        )
    )
)

## OCP/MPC Options

#### Set solver and discretization options

In [ ]:
tc.set_ocp_solver("ipopt")

disc_settings = {
    "discretization method": "multiple shooting",
    "horizon size": horizon_size,
    "order": 1,
    "integration": "rk",
}
tc.set_discretization_settings(disc_settings)

#### Set parameter values
In the defined task context, the parameters correspond to the initial state of the robot.

In [ ]:
tc.ocp.set_value(q0, q_init)
tc.ocp.set_value(q_dot0, [0] * robot.ndof)
tc.ocp.set_value(s0, 0)
tc.ocp.set_value(s_dot0, 0)

#### Solve the OCP that describes the task

In [ ]:
sol = tc.solve_ocp()

#### MPC simulation

Create world simulator based on pybullet

In [ ]:
from tasho import world_simulator
import pybullet as p

obj = world_simulator.world_simulator(bullet_gui=False)

Add robot to the world environment

In [ ]:
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
yumiID = obj.add_robot(position, orientation, "yumi")

Determine number of samples that the simulation should be executed

In [ ]:
no_samples = int(t_mpc / obj.physics_ts)

Correspondence between joint numbers in bullet and OCP

In [ ]:
joint_indices = [11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Begin the visualization by applying the initial control signal

In [ ]:
obj.resetJointState(yumiID, joint_indices, q_init)
obj.setController(yumiID, "velocity", joint_indices, targetVelocities=q_dot_init)

#### Define MPC parameters

In [ ]:
mpc_params = {"world": obj}

In [ ]:
q0_params_info = {
    "type": "joint_position",
    "joint_indices": joint_indices,
    "robotID": yumiID,
}
q_dot0_params_info = {
    "type": "joint_velocity",
    "joint_indices": joint_indices,
    "robotID": yumiID,
}
s0_params_info = {"type": "progress_variable", "state": True}
s_dot0_params_info = {"type": "progress_variable", "state": True}

mpc_params["params"] = {
    "q0": q0_params_info,
    "q_dot0": q_dot0_params_info,
    "s0": s0_params_info,
    "s_dot0": s_dot0_params_info,
    "robots": {yumiID: robot},
}
mpc_params["log_solution"] = True

Replace the solver used in the task context (IPOPT) with the SQP method and QRQP for MPC execution

In [ ]:
mpc_params["disc_settings"] = disc_settings
mpc_params["solver_name"] = "sqpmethod"
mpc_params["solver_params"] = {"qrqp": True}
mpc_params["t_mpc"] = t_mpc
mpc_params["control_type"] = "joint_velocity"  #'joint_torque'
mpc_params["control_info"] = {
    "robotID": yumiID,
    "discretization": "constant_acceleration",
    "joint_indices": joint_indices,
    "no_samples": no_samples,
}

#### Define code-generation options
This options may include just-in-time compilation, compilation flags, selection of desired compiler, etc.

In [ ]:
mpc_params["codegen"] = {
    "codegen": False,
    "filename": "mpc_c",
    "compilation": False,
    "compiler": "gcc",
    "flags": "-O3 -ffast-math -flto -funroll-loops -march=native -mfpmath=both -mvzeroupper",
    "use_external": False,
    "jit": False,
}

#### Create monitor
Add monitor to check some termination criteria based on the path-progress variable $s$

In [ ]:
tc.add_monitor(
    {
        "name": "termination_criteria",
        "expression": s,
        "reference": 0.99,
        "greater": True,
        "initial": True,
    }
)

#### MPC object
Initialize MPC object based on the task context created previously and the defined MPC parameters

In [ ]:
sim_type = "bullet_notrealtime"
mpc_obj = MPC.MPC(tc, sim_type, mpc_params)
mpc_obj.max_mpc_iter = 4000

Run the OCP with IPOPT once to get a good initial guess for the MPC, and code-generate the MPC function if required

In [ ]:
mpc_obj.configMPC_fromcurrent()

#### Execute the MPC loop

In [ ]:
mpc_obj.runMPC()

The MPC execution is finished when the termination criteria defined by the monitor is reached.